In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pickle
import itertools
import tqdm
# import hypergrad as hg
from mlmodel import *
import utils
import numpy as np
from sparsemax import Sparsemax
from argparse import ArgumentParser
from ZSTL_model import ZSTL
import pandas as pd
import itertools

In [2]:
net = FuncRecursiveNet([
    FLinearLayer(1, True)
])
shape_record = {0: [(1, 2049)]}

In [3]:
kb_size = 5
train_size = 35

In [4]:
filename = '../ZSTL_Data/Animals_with_Attributes2/splitedTask/task_train_data_standard.pickle'
#filename = '../ZSTL_Data/CUB_200_2011/CUB_200_2011/splitedTask/task_train_data_standard.pickle'
with open(filename, 'rb') as f:
    dataset = pickle.load(f)
print(len(dataset), len(list(dataset.keys())))
print(type(dataset))

total_len = len(dataset)
test_size = total_len - kb_size - train_size


50 50
<class 'dict'>


In [5]:
#check stored data correctness
def check_attr_correct(num_task, dataset, task_attr_byID, model):
    task_train_data = {}
    task_test_data = {}
    task_val_data = {}
    for t in range(num_task):
        cur_attr = dataset[t+1][0]
        cur_param = utils.toTensor(dataset[t+1][1])
        cur_x = utils.toTensor(dataset[t+1][2])
        cur_y = dataset[t+1][3]
        print((cur_attr==task_attr_byID[t+1]).all())


        reshaped_w = utils.reshape_w(cur_param, shape_record)
        pred = model(reshaped_w, cur_x)
        pred = torch.sigmoid(pred)
        pred[pred>=0.5] = torch.ones_like(pred[pred>=0.5])
        pred[pred<0.5] = torch.zeros_like(pred[pred<0.5])
        print(np.sum(utils.toNumpy(pred)==cur_y)/100)



In [6]:
path_attributes = '../ZSTL_Data/Animals_with_Attributes2/predicate-matrix-continuous.txt'
def gen_attr(path_attributes):
    data_attributes = pd.read_csv(path_attributes, sep="\n", header=None)
    print(data_attributes)
    lst = []
    task_attr_byID = {}
    i = 1
    for r in data_attributes.iterrows():
        # print(len(r), )
        s = r[1].to_numpy()
        # print(len(s[0].split()), s[0].split())
        print(len(s[0].split()))
        task_attr_byID[i] = np.array([float(a) for a in s[0].split()])
        print(task_attr_byID[i].shape)
        i += 1
    return task_attr_byID

#task_attr_byID = gen_attr(path_attributes)
#check_attr_correct(total_len, dataset, task_attr_byID, net)


In [7]:
np.random.seed(666)
indx = [ x+1 for x in range(len(dataset))]
support_indx = list(np.random.choice(indx, size=kb_size, replace=False))
print(len(support_indx))
temp = [x for x in indx if x not in support_indx]
train_indx = list(np.random.choice(temp, size=train_size, replace=False))
temp = [x for x in temp if x not in train_indx]
print(len(train_indx))
test_indx = temp
print(len(test_indx))


support_data = utils.Dataset([dataset[d] for d in support_indx])
train_data = utils.Dataset([dataset[d] for d in train_indx])
test_data = utils.Dataset([dataset[d] for d in test_indx])
support_loader = DataLoader(support_data, batch_size=kb_size, shuffle=False)
train_loader = DataLoader(train_data, batch_size=train_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=test_size, shuffle=True)

5
35
10


In [8]:
support_a, support_w, support_x, support_y = next(iter(support_loader))
support_a, support_w, support_x, support_y = support_a.float(), support_w.float(), support_x.float(), support_y.float()
support_a = support_a.squeeze().t()
support_w = support_w.squeeze().t()
dm = support_a.size()[0]
print(dm)
d  = support_w.size()[0]
print(d)

param_dict = {}
param_dict['rho'] = 1e-5
param_dict['mu'] = 1e-5
param_dict['loss'] = 'binary class'
param_dict['outer lr'] = 1e-3
param_dict['align lr'] = 1e-4
param_dict['dm'] = dm
param_dict['d'] = d
param_dict['model_shape'] = shape_record
param_dict['atten_activation'] = 'Sparsemax'
print(param_dict)


ZSTL_model = ZSTL(support_w, support_a, support_x, net, param_dict)
ZSTL_model.train(train_loader, test_loader, max_iter=500)


85
2049
{'rho': 1e-05, 'mu': 1e-05, 'loss': 'binary class', 'outer lr': 0.001, 'align lr': 0.0001, 'dm': 85, 'd': 2049, 'model_shape': {0: [(1, 2049)]}, 'atten_activation': 'Sparsemax'}
init mean test metric 0.5980000000000001; align loss 0.28833961486816406
1/500 o_loss 2.3319798475561715; m train metric 0.5541428571428572; m test metric 0.623; align loss  0.2568095326423645
10/500 o_loss 1.07951808829717; m train metric 0.6971428571428571; m test metric 0.6925000000000001; align loss  0.25673040747642517
20/500 o_loss 0.9048650095831753; m train metric 0.7311428571428572; m test metric 0.7154999999999999; align loss  0.25661975145339966
30/500 o_loss 0.7600210903561674; m train metric 0.7484285714285716; m test metric 0.7100000000000002; align loss  0.2565110921859741
40/500 o_loss 0.648300460991907; m train metric 0.7994285714285716; m test metric 0.7155000000000001; align loss  0.2564249634742737
50/500 o_loss 0.5743968977642778; m train metric 0.8138571428571426; m test metric 0.6

KeyboardInterrupt: 

In [8]:



indx = train_indx
val_indx = list(np.random.choice(indx, size=20, replace=False))
print(len(val_indx))
train_indx_splited = [x for x in indx if x not in val_indx]
print(len(train_indx_splited))

val_data = utils.Dataset([dataset[d] for d in val_indx])
train_splited_data = utils.Dataset([dataset[d] for d in train_indx_splited])
val_loader = DataLoader(val_data, batch_size=len(val_indx), shuffle=True)
train_splited_loader = DataLoader(train_splited_data, batch_size=len(train_indx_splited), shuffle=True)

best_hp = utils.hp_select_binClass(train_splited_loader, val_loader, support_loader, d, dm, net, shape_record)
print('best_hp ', best_hp)

312
2049
20
120
120
20
num of hp  16
rho for w_kb 0.1; mu for a_kb 0.1;
init mean test metric 0.5762499999999999; align loss 258.3763427734375
1/1500 o_loss 17.96778246735533; m train metric 0.613333333333333; m test metric 0.6143749999999999; align loss  250.96774291992188


KeyboardInterrupt: 

In [10]:


param_dict = {}
param_dict['rho'] = best_hp['rho']
param_dict['mu'] = best_hp['mu']
param_dict['loss'] = 'binary class'
param_dict['outer lr'] = 1e-3
param_dict['align lr'] = 1e-4
param_dict['dm'] = dm
param_dict['d'] = d
param_dict['model_shape'] = shape_record
param_dict['atten_activation'] = 'Sparsemax'
print(param_dict)


ZSTL_model = ZSTL(support_w, support_a, support_x, net, param_dict)
ZSTL_model.train(train_loader, test_loader, max_iter=1800)


2049 312
